**Imports**

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt

**Constants**

In [3]:
################################################################################
# DEFINE CONSTANTS
################################################################################
# Internal Units
# Length: m
# Mass: kg
# Time: s
# Temperature: K
# Amount: mol

Rg = 8.314 #J/mol/K

min_to_s = 60 #seconds
L_to_m3 = 0.001 #m3
m3_to_L = 1000 #L

**Physical Properties**

PA  = Phthalic Anhydride \
B   = Butanol \
MBP = Monobutyl Phthalate \
DBP = Dibutyl Phthalate \
W   = Water \
DBE = Dibutyl Ether \
SA  = Sulfuric Acid 

In [13]:
# Molecular Weights from NIST
# Units g/mol or kg/kmol
PA_MW = 148.1156
B_MW = 74.1216
MBP_MW = 222.2372
DBP_MW = 278.3435
W_MW = 18.0153
DBE_MW = 130.2279
SA_MW = 98.078

**Mass Balance**

We want to produce 63 ktonnes per year of DBP

In [16]:
molar_flow_per_hour = 63 * 1000 * 1000 / 351 / 278.3435 / 24 # ktonnes/yr to kmol/hr
print(molar_flow_per_hour, "kmol/hr")

26.86835682756191 kmol/hr


**Reaction Network**

In [4]:
species = ['PA', 'B', 'MBP', 'DBP', 'DBE', 'W']
nRxns = 3
nSpecies = len(species)

nu = np.zeros((nSpecies, nRxns))

In [17]:
# R1: PA + B -> MBP
nu[0,0] = -1 #PA
nu[1,0] = -1 #B
nu[2,0] = +1  #MBP

# R2: MBP + B <-> DBP + W
nu[1, 1] = -1 #B
nu[2, 1] = -1 #MBP
nu[3, 1] = +1 #DBP
nu[5, 1] = +1 #W

# R3: 2B -> DBE + W
nu[1, 2] = -2 #B
nu[4, 2] = +1 #DBE
nu[5, 2] = +1 #W

print(nu)

[[-1.  0.  0.]
 [-1. -1. -2.]
 [ 1. -1.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  1.  1.]]


**Kinetic Parameters**

In [6]:
# R1: PA + B -> MBP
# No data given, treating as fast, irreversible

# R2: MBP + B <-> DBP + W
### Forward
E2f = 89.97e3  # J/mol
A2f = 3.663e10 # 1/(M^3 * min)

### Backward
E2b = 92.03e3  # J/mol
A2b = 8.480e10 # 1/(M^3 * min)

# R3: 2B -> DBE + W
E3 = 98.5e3    # J/mol
A3 = 2.45e8    # 1/(M^2 * min)

**Rate Constants**

In [7]:
def k_values(T):
    """
    Return k2f, k2b, k3 at temperature T [K].
    Units: 1/(M^order * s)
    """
    k2f = (A2f * np.exp(-E2f / (Rg * T))) / min_to_s
    k2b = (A2b * np.exp(-E2b / (Rg * T))) / min_to_s
    k3  = (A3  * np.exp(-E3  / (Rg * T)))  / min_to_s
    return k2f, k2b, k3

In [18]:
k_values(130 + 273.15)

(np.float64(0.0013433455030103338),
 np.float64(0.0016820175087866414),
 np.float64(7.051422363864068e-07))

In [8]:
def rxn_rates(C, T, C_SA):
    """
    Reaction rates r = [r1, r2, r3] in M/s.

    C    = [PA, B, MBP, DBP, DBE, W]  (mol/L)
    T    = temperature [K]
    C_SA = sulfuric acid concentration [M]
    """
    C_PA, C_B, C_MBP, C_DBP, C_DBE, C_W = C
    k2f, k2b, k3 = k_values(T)

    # Rxn 1: PA + B -> MBP, treated as instantaneous in feed
    r1 = 0.0

    # Rxn 2: MBP + B <-> DBP + W
    r2f = k2f * C_SA * C_B   * (C_MBP**2)
    r2b = k2b * C_SA * C_MBP * C_DBP * C_W
    r2  = r2f - r2b

    # Rxn 3: B -> DBE   (empirical law uses B^2 and SA)
    r3  = k3  * C_SA * (C_B**2)

    return np.array([r1, r2, r3])

In [9]:
def effective_feed(C_PA0, C_B0, C_MBP0, C_DBP0, C_DBE0, C_W0):
    """
    Apply Rxn 1 as instantaneous & complete (PA + B -> MBP)
    before the kinetic reactions 2 and 3.

    Inputs: true feed concentrations [M].
    Returns: effective inlet concentrations [M] to the CSTR.
    """
    if C_B0 < C_PA0:
        raise ValueError("Need B in excess for Rxn 1 completion.")

    C_PA_in  = 0.0
    C_MBP_in = C_MBP0 + C_PA0
    C_B_in   = C_B0   - C_PA0

    return np.array([C_PA_in, C_B_in, C_MBP_in, C_DBP0, C_DBE0, C_W0])

In [10]:
############################################################
# CSTR model
############################################################

def cstr_residual(C, C_in, tau, T, C_SA):
    """
    Steady-state CSTR balances.

    C     : outlet concentrations [M]
    C_in  : inlet concentrations [M]
    tau   : residence time [s]
    T     : temperature [K]
    C_SA  : acid concentration [M]
    """
    r_vec = rxn_rates(C, T, C_SA)     # (3,)
    R_net = nu @ r_vec                # (6,)
    return C_in - C + tau * R_net

def solve_cstr(C_in, tau, T, C_SA):
    C_guess = C_in.copy()
    C_out   = opt.fsolve(cstr_residual, C_guess,
                         args=(C_in, tau, T, C_SA))
    return C_out


In [19]:
def wtpercent_to_C_SA(wt_SA, rho_liq=1100.0):
    """
    Convert sulfuric acid loading from wt% (of 98 wt% H2SO4)
    to mol/L of H2SO4 in the liquid phase.

    wt_SA : mass% of commercial 98 wt% H2SO4 in liquid (1–4)
    rho_liq : liquid density [g/L], default ~1.1 g/mL = 1100 g/L

    Returns C_SA [mol/L].
    """
    # mass of commercial acid per L [g/L]
    mass_acid_solution = (wt_SA / 100.0) * rho_liq
    # mass of pure H2SO4 per L
    mass_H2SO4 = 0.98 * mass_acid_solution
    # mol/L
    C_SA = mass_H2SO4 / SA_MW
    return C_SA